In [196]:
!pip install beautifulsoup4

In [197]:
!pip install lxml

In [198]:
!pip install html5lib

In [316]:
from bs4 import BeautifulSoup

In [317]:
import requests

In [318]:
with open('codes.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [399]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [400]:
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [401]:
My_table = soup.find('table',{'class':'wikitable sortable'})

In [402]:
Postcode = []

for row in My_table.findAll("tr"): #tr is one row
    cells = row.findAll("td") #td is the input in each cell
    if len(cells) == 3: #table has three columns
        Postcode.append(cells[0].find(text=True)) #Postcode

In [403]:
Neighborhood = []
for row in My_table.findAll("tr"): #tr is one row
    cells = row.findAll("td") #td is the input in each cell
    if len(cells) == 3: #table has three columns
        Neighborhood.append(cells[2].findAll(text=True))

In [404]:
Borough = []
for row in My_table.findAll("tr"): #tr is one row
    cells = row.findAll("td") #td is the input in each cell
    if len(cells) == 3: #table has three columns
        Borough.append(cells[1].find(text=True))


In [405]:
df = pd.DataFrame()
df['Postcode'] = Postcode

In [406]:
df['Borough'] = Borough

In [407]:
df['Neighborhood'] = Neighborhood

In [408]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,[Not assigned ]
1,M2A,Not assigned,[Not assigned ]
2,M3A,North York,"[Parkwoods, ]"
3,M4A,North York,"[Victoria Village, ]"
4,M5A,Downtown Toronto,"[Harbourfront, ]"


In [409]:
type(df['Neighborhood'][0]) #check the type of the values in the Neighborhood column

bs4.element.ResultSet

In [410]:
df['Neighborhood'] = df['Neighborhood'].astype(str) # convert the Neighborhood olumn to string

In [411]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,['Not assigned\n']
1,M2A,Not assigned,['Not assigned\n']
2,M3A,North York,"['Parkwoods', '\n']"
3,M4A,North York,"['Victoria Village', '\n']"
4,M5A,Downtown Toronto,"['Harbourfront', '\n']"


In [412]:
df.Neighborhood = df.Neighborhood.str[:-3] #remove unecessary part of the column

In [413]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,['Not assigned\
1,M2A,Not assigned,['Not assigned\
2,M3A,North York,"['Parkwoods', '\"
3,M4A,North York,"['Victoria Village', '\"
4,M5A,Downtown Toronto,"['Harbourfront', '\"


In [414]:
import string
df.Neighborhood = df.Neighborhood.apply(lambda x: x.translate(str.maketrans('','',string.punctuation))) #further remove unecessary parts of the column

In [415]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [416]:
df['Borough'] = df['Borough'].astype(str) # convert the Borough olumn to string

In [417]:
df['Postcode'] = df['Postcode'].astype(str) # convert the Postcode column to string

In [418]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [419]:
#Remove white spaces
df['Borough'] = df['Borough'].apply(lambda x: x.strip())
df['Postcode'] = df['Postcode'].apply(lambda x: x.strip())
df.Neighborhood = df.Neighborhood.apply(lambda x: x.strip())

In [420]:
df.head(6)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [421]:
df.head(6)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [422]:
#copy data
df2 = df.copy()

In [423]:
df2 =df2.groupby('Postcode')['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index() #GROUPING DATA BY Postcode


In [424]:
df2.loc[df2['Postcode'] == 'M5A'] #checking to see if code works

,Postcode,Neighborhood
80,M5A,"Harbourfront, Regent Park"


In [425]:
len(df2)

180

In [426]:
#merge data
df3 = df2.merge(df, left_on='Postcode', right_on='Postcode', how = 'inner')

In [427]:
df3.loc[df3['Postcode'] == 'M5A'] #checking to see if code works

,Postcode,Neighborhood_x,Borough,Neighborhood_y
118,M5A,"Harbourfront, Regent Park",Downtown Toronto,Harbourfront
119,M5A,"Harbourfront, Regent Park",Downtown Toronto,Regent Park


In [428]:
df4 = df3[~df3[['Postcode', 'Neighborhood_x']].apply(frozenset, axis=1).duplicated()]

In [429]:
# remove duplicates in Postcode column
#df3.drop_duplicates(subset ="Postcode", keep = False, inplace = True) 

In [434]:
df4.head()

,Postcode,Neighborhood_x,Borough,Neighborhood_y
0,M1A,Not assigned,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough,Rouge
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,Highland Creek
6,M1E,"Guildwood, Morningside, West Hill",Scarborough,Guildwood
9,M1G,Woburn,Scarborough,Woburn


In [435]:
df4.loc[df4['Postcode'] == 'M5V'] #checking to see if code works

,Postcode,Neighborhood_x,Borough,Neighborhood_y
148,M5V,"CN Tower, Bathurst Quay, Island airport, Harbo...",Downtown Toronto,CN Tower


In [436]:
#remove Neighborhood_y column
df4 = df4.drop(columns='Neighborhood_y')


In [437]:
#Rename Neighborhood_X
df4.rename(columns={'Neighborhood_x':'Neighborhood'}, inplace=True)

In [438]:
df4.head()

,Postcode,Neighborhood,Borough
0,M1A,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
6,M1E,"Guildwood, Morningside, West Hill",Scarborough
9,M1G,Woburn,Scarborough


In [439]:
finaldata = df4

In [440]:
finaldata.head()

,Postcode,Neighborhood,Borough
0,M1A,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
6,M1E,"Guildwood, Morningside, West Hill",Scarborough
9,M1G,Woburn,Scarborough


In [442]:
finaldata.shape

(180, 3)

In [ ]:
#Dont forget to 